In [0]:
import os
import tensorflow
from PIL import ImageFile, Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array
import glob
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, BatchNormalization, LSTM

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 10
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 13650
# Количество изображений для проверки
nb_validation_samples = 2925
# Количество изображений для тестирования
nb_test_samples = 2925

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [0]:
X = []
filenames = []
c = 0
for filename in sorted(glob.glob('/content/drive/My Drive/train/*.jpg')):
  image = cv2.imread(filename)
  image = cv2.resize(image, (img_width, img_height))
  filenames.append(filename)
  X.append(image)
  c += 1
  print(c)
print(X[0])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
X = np.divide(X, 255.0)

In [0]:
Y = pd.read_csv('/content/drive/My Drive/training.csv', sep=',')
Y = Y[['subDirectory_filePath', 'valence', 'arousal']]
Y['subDirectory_filePath'] = Y['subDirectory_filePath'].apply(lambda x: x.split("/")[1])
Y = Y.sort_values('subDirectory_filePath')

In [0]:
Y.head()

,subDirectory_filePath,valence,arousal
34738,0000792211b64f6a59bd2d95fee49eabe6373ec1d88f22...,-0.488145,0.821877
157036,0000f8a4575c15055a9ee0a72c9aa5bf9ac00558173565...,-0.316056,-0.136957
26740,00012312112a15995f1d1c1ac640db7191eacab8099e90...,0.710109,0.689223
178824,0001636b7a16a63b2d9c8f5e4b7be02e4841d3e0af3ebd...,-0.420635,0.515873
298470,0001720743cf22095bd3b2c94b35f244faf47545e26168...,0.521274,0.126796


In [0]:
# Y1 = []
# for index, row in Y.iterrows():
#   for f in filenames:
#     if f.find(row['subDirectory_filePath']) >= 0:
#       Y1.append(row)
#       print("appending " + str(row))
#       break

In [0]:
df = pd.read_csv('/content/drive/My Drive/y_modified.csv', sep=',')
df = df[['subDirectory_filePath', 'valence', 'arousal']]
df

,subDirectory_filePath,valence,arousal
0,000c10a8f6735212d71eb54298bc47ea56dc08e64ed2ca...,0.004313,-0.008627
1,001fffed9707757082906ff2eb89cad6ddf242195fe6a5...,0.539683,0.023810
2,002b6db57713be4a264d8dd13ee543b031de7577af983c...,-0.193573,0.590399
3,003e5febbe7808b02f6cafe17878cdfd59851726451209...,0.526805,-0.036153
4,004b10f6bfbce7f58d124ac653451bdd975a7113d20b18...,0.849206,0.238095
...,...,...,...
2137,7a2b38adf6aa2d7b8b93d02f5defb7a6aebd738f8e544e...,0.380952,0.063492
2138,7a2e5d157c9cbe119103f1a60052350ef3cbd14f651628...,-0.111111,-0.920635
2139,7a7b4f6fb23aaa9189a6faf644d07780947b00048d6992...,0.552105,0.219979
2140,7a7ea6d03d1cf4275e6dfc6d26104467c079edccecfcb0...,0.711698,0.125086


In [0]:
d = dict(zip(filenames, X))
list(d.keys())[0]

'/content/drive/My Drive/train/000c10a8f6735212d71eb54298bc47ea56dc08e64ed2cab8913d683f.jpg'

In [0]:
files = df['subDirectory_filePath']
new_dict = {}
for key in d.keys():
  for file in files:
    if key.find(file) >= 0:
      new_dict.update({key:d[key]})
      break
print(len(new_dict))

2142


In [0]:
df['image'] = list(new_dict.values())

In [0]:
indexes_to_drop = df[(df['valence'] == -2) | (df['arousal'] == -2)].index
df.drop(indexes_to_drop, inplace=True)

In [0]:
# model for valence
model_v = Sequential()
model_v.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model_v.add(Activation('relu'))
model_v.add(BatchNormalization(axis=-1))
model_v.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model_v.add(Conv2D(32, (3, 3)))
model_v.add(Activation('relu'))
model_v.add(BatchNormalization(axis=-1))
model_v.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model_v.add(Conv2D(64, (3, 3)))
model_v.add(Activation('relu'))
model_v.add(BatchNormalization(axis=-1))
model_v.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model_v.add(Conv2D(64, (3, 3)))
model_v.add(Activation('relu'))
model_v.add(BatchNormalization(axis=-1))
model_v.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model_v.add(Conv2D(128, (3, 3)))
model_v.add(Activation('relu'))
model_v.add(BatchNormalization(axis=-1))
model_v.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model_v.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model_v.add(Dense(100))
model_v.add(Activation('relu'))
model_v.add(BatchNormalization(axis=-1))
model_v.add(Dropout(0.5))
model_v.add(Dense(1, activation='linear'))

model_v.summary()

model_v.compile(loss='mean_squared_error', optimizer='sgd')








Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_22 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 148, 148, 32)      128       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_23 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 72, 72, 32)

In [0]:
# model for arousal
model_a = Sequential()
model_a.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model_a.add(Activation('relu'))
model_a.add(BatchNormalization(axis=-1))
model_a.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model_a.add(Conv2D(32, (3, 3)))
model_a.add(Activation('relu'))
model_a.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model_a.add(Conv2D(64, (3, 3)))
model_a.add(Activation('relu'))
model_a.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_last"))

model_a.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model_a.add(Dense(50))
model_a.add(Activation('relu'))
model_a.add(BatchNormalization(axis=-1))
model_a.add(Dropout(0.5))
model_a.add(Dense(4, activation='relu'))
model_a.add(Dense(1, activation='linear'))

model_a.summary()

model_a.compile(loss='mean_squared_error', optimizer='sgd')

In [0]:
x_train_v, x_test_v, y_train_v, y_test_v = train_test_split(np.stack(list(df['image']), axis=0), list(df['valence']), test_size=0.33)

model_v.fit(x_train_v, y_train_v, epochs=200, batch_size=64, validation_data=(x_test_v, y_test_v))



Train on 1179 samples, validate on 581 samples
Epoch 1/200
1179/1179 [==============================] - 9s 7ms/step - loss: 2.2422 - val_loss: 0.7328
Epoch 2/200
1179/1179 [==============================] - 1s 1ms/step - loss: 0.6132 - val_loss: 0.2945
Epoch 3/200
1179/1179 [==============================] - 1s 1ms/step - loss: 0.3790 - val_loss: 0.2519
Epoch 4/200
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2878 - val_loss: 0.2404
Epoch 5/200
1179/1179 [==============================] - 1s 992us/step - loss: 0.2414 - val_loss: 0.2304
Epoch 6/200
1179/1179 [==============================] - 1s 996us/step - loss: 0.2171 - val_loss: 0.2354
Epoch 7/200
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2156 - val_loss: 0.2295
Epoch 8/200
1179/1179 [==============================] - 1s 1ms/step - loss: 0.2073 - val_loss: 0.2288
Epoch 9/200
1179/1179 [==============================] - 1s 996us/step - loss: 0.2181 - val_loss: 0.2276
Epoch 10/200
1179/

In [0]:
x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(np.stack(list(df['image']), axis=0), list(df['arousal']), test_size=0.33)

model_a.fit(x_train_a, y_train_a, epochs=100, batch_size=128, validation_data=(x_test_a, y_test_a))

In [0]:
# make predictions on the testing data
preds = model.predict(x_test)
 
# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - y_test
percentDiff = (diff / y_test) * 100
absPercentDiff = np.abs(percentDiff)

In [0]:
model_json_v = model_v.to_json()
# Записываем модель в файл
json_file = open("affect_model_v.json", "w")
json_file.write(model_json_v)
json_file.close()

model_v.save_weights("affect_model_v.h5")

In [0]:
model_json_a = model_a.to_json()
# Записываем модель в файл
json_file = open("affect_model_a.json", "w")
json_file.write(model_json_a)
json_file.close()

model_a.save_weights("affect_model_a.h5")

In [0]:
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

print('start loading image...')
img = image.load_img("hehehe.jpg", target_size=(150, 150))
print('start creating array from image...')
x = image.img_to_array(img)
print('start x /= 255...')
x /= 255
print('start expanding x...')
x = np.expand_dims(x, axis=0)
print('openning json...')
json_file_v = open("affect_model_v.json", "r")
json_file_a = open("affect_model_a.json", "r")
print('reading json...')
loaded_model_json_v = json_file_v.read()
loaded_model_json_a = json_file_a.read()
print('closing json...')
json_file_v.close()
json_file_a.close()
print('creating model from json...')
loaded_model_v = model_from_json(loaded_model_json_v)
loaded_model_a = model_from_json(loaded_model_json_a)
print('loading weights from cats_vgg19_model.h5...')
loaded_model_v.load_weights("affect_model_v.h5")
loaded_model_a.load_weights("affect_model_a.h5")
print('compiling weights from cats_vgg19_model.h5...')
loaded_model_v.compile(loss='mean_squared_error', optimizer='sgd')
loaded_model_a.compile(loss='mean_squared_error', optimizer='sgd')
print('getting model prediction from x...')
prediction_v = loaded_model_v.predict(x)
prediction_a = loaded_model_a.predict(x)
print('creating array of breeds...')
print("valence: ", prediction_v)
print("arousal: ", prediction_a)